# TP Coding Convolutional Neural Networks in tensorflow/keras

Author : Alasdair Newson

alasdair.newson@telecom-paris.fr

## Objective:

We want to implement a Convolutional Neural Network (CNN) to do image classification. For this we will use two well-known image datasets :

###MNIST

Mnist is a dataset of 60,000 28$\times$28 greyscale images of handwritten digits.

### CIFAR 10

CIFAR-10 dataset https://www.cs.toronto.edu/~kriz/cifar.html.

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

### Your task:
You need to add the missing parts in the code (parts between # --- START CODE HERE and # --- END CODE HERE or # FILL IN CODE)

# Load packages

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
print(tf.keras.__version__)


## Import data

We are going to start with the mnist dataset. The mnist dataset has 10 classes which go from "0" to "9" (all the digits). We are going to truncate the number of elements in the database for faster training. Note that "scalar" means that the class is encoded as a scalar, rather than as a one-hot vector (we do the conversion afterwards).

In [ ]:

from keras.datasets import mnist
(X_train, Y_train_scalar), (X_test, Y_test_scalar) = mnist.load_data()

n_max = 5000
X_train = X_train[0:n_max,:,:]
X_test = X_test[0:n_max,:,:]
Y_train_scalar = Y_train_scalar[0:n_max]
Y_test_scalar = Y_test_scalar[0:n_max]

mnist_label_list = [ '0', '1','2','3','4','5','6','7','8','9']

print(X_train.shape)
print(X_test.shape)
print(np.unique(Y_train_scalar))

Now, let's look at some of the data :

In [ ]:
plt.figure(figsize=(10, 6))
for idx in range(0,10):
    plt.subplot(2, 5, idx+1)
    rand_ind = np.random.randint(0,X_train.shape[0])
    plt.imshow(X_test[rand_ind,:,:],cmap='gray')
    plt.title(mnist_label_list[int(Y_test_scalar[rand_ind])])


Now, we carry out some data pre-processing. In particular, we normalise the data to the range $[0,1]$ and convert the labels to a one-hot encoding :

In [ ]:
# input image dimensions (mnist data is grey-level)
img_rows, img_cols, nb_channels = X_train.shape[1], X_train.shape[2], 1

# In case depth is 1 (black and white pictures) -> reshape to proper format
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, nb_channels)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, nb_channels)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

Y_train = to_categorical(Y_train_scalar)
Y_test = to_categorical(Y_test_scalar)

# number of classes
nb_classes = Y_train.shape[1]

print('X_train shape:', X_train.shape)
#print('y_train shape:', y_train.shape)
print('Y_train shape:', Y_train.shape)
print('There are {} train data'.format(X_train.shape[0]))
print('There are {} test data'.format(X_test.shape[0]))
print(img_rows, img_cols, nb_channels)


## Defining the model for MNIST

We will now define the simple CNN described below, for use with MNIST. The input of the CNN is a set of (28,28,1) image tensors. We apply :

    - a Convolutional layer of 32 filters of shape (3,3), with stride (1,1) and padding='same'
    - a ReLu activation function
    
    - a Convolutional layer of 32 filters of shape (3,3), with stride (1,1) and padding='same'
    - a ReLu activation function
    - a Max Pooling Layer of shape (2,2) and stride (2,2) (i.e. we reduce by two the size in each dimension)
    
    - We then Flatten the data (reduce them to a vector in order to be able to apply a Fully-Connected layer to it)
    - A softmax activation function which outputs are the $P(y_c | X)$ (multi-class problem)

We define the parameters of the model :

In [ ]:
learning_rate = 0.01
n_epochs = 10
batch_size = 64

# number of convolutional filters to use
nb_filters = 32
# convolution kernel size
kernel_size = (3, 3)
# size of pooling area for max pooling
pool_size = (2, 2)

# --- Size of the successive layers
n_h_0 = nb_channels  # number of input channels
n_h_1 = nb_filters
n_h_2 = nb_filters

## Creating the CNN model using the Sequential API

Tensorflow has a simple way of adding layers to create a neural network. First, you can indicate to Tensorflow that the model is 'sequential', that is to say, a straight-forward CNN. For this, you can use the following function :
- ```model = Sequential()```

After this, you can add layers with the function.

- ```model.add()```

You can then use the ```Conv2D```, ```Activation```, ```MaxPooling2D```, ```Flatten``` and ```Dense``` (fully connected) functions to specify different layer types. Note that in the case of this approach, you will have to specify the input image size in the first layer of the network. So, for example, if the first layer is convolutional :

- model = Sequential()
- model.add(Conv2D(nb_filters, kernel_size, input_shape=input_shape, name='Conv1'))


## Creating the CNN model using the standard API

Otherwise, another approach to creating the model is to explicitly create the input variable, and just cascade the different functions, as in Tensorflow. So, for the same example, we would have :

- input = Input(shape=(img_rows,img_cols,nb_channels))
- output = Conv2D(input_shape=self.img_shape,filters=nb_filters,kernel_size=kernel_size)(input)
- model = Model(input, output)

Create your CNN now with the network parameters specified above :

In [ ]:
# --- START CODE HERE

model = ...

# --- END CODE HERE

Now, you need to :

- compile
- display (```summary```)
- train the model

and show the test accuracy once the training is finished. You should use the ```optimizers.Adam``` optimiser, and the correct loss (which one ?).

In [ ]:
# compile, train and evaluate model
# BEGIN STUDENT CODE

# compile model

# display model

# fit model

# END STUDENT CODE

score = model.evaluate(X_test, Y_test, verbose=False)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

You should achieve a test accuracy of around 0.94 within 10 epochs.

This is pretty good ! However, don't get too excited just yet. Unfortunately, one of the golden rules of deep learning is that everything works with mnist (unless you have gotten the model really wrong). So often, mnist is just used as a sanity check. Let's now look at a more complex dataset : CIFAR10

## 2/ CNN on CIFAR10

Cifar10 has ten categories, which are much more complex than the digits in mnist. In this case, we are going to add more layers to increase the capacity of the network. We create the following network :

    - a Convolutional layer of 32 filters of shape (3,3), with stride (1,1) and padding='same'
    - a ReLu activation function
    
    - a Convolutional layer of 32 filters of shape (3,3), with stride (1,1) and padding='same'
    - a ReLu activation function
    - a Max Pooling Layer of shape (2,2) and stride (2,2) (i.e. we reduce by two the size in each dimension)
    
    - a Convolutional layer of 32 filters of shape (3,3), with stride (1,1) and padding='same'
    - a ReLu activation function
    - a Max Pooling Layer of shape (2,2) and stride (2,2) (i.e. we reduce by two the size in each dimension)
    
    - We then Flatten the data (reduce them to a vector in order to be able to apply a Fully-Connected layer to it)
    - A softmax activation function which outputs are the $P(y_c | X)$ (multi-class problem)


As previsously, we investigate the data, display some samples, and we carry out some pre-processing.

### Import data

In [ ]:
from tensorflow.keras.datasets import cifar10
(X_train, Y_train_scalar), (X_test, Y_test_scalar) = cifar10.load_data()

print("Train data shape : ", X_train.shape)
print("Test data shape : ", X_test.shape)
print("Number of data categories : ", np.unique(Y_train_scalar))

The CIFAR-10 dataset has 10 classes. These are the following :

In [ ]:
cifar_10_list = [ 'airplane', 'automobile','bird','cat','deer','dog','frog','horse','ship','truck']

### Display some of the images

In [ ]:
plt.figure(figsize=(10, 6))
for idx,i in enumerate(range(100,110)):
    plt.subplot(2, 5, idx+1)
    plt.imshow(X_train[i, :, :, :])
    plt.title(cifar_10_list[int(Y_train_scalar[i])])

As you can see, the images are in quite small resolution. This makes them more easy to handle computationally, however it also means that they are quite difficult to analyse, even for a human being. Therefore, __you might not get that great accuracy scores__. The goal is not to get great scores (this would take longer training and potentially a larger network).

## Format conversion

In [ ]:
# input image dimensions
img_rows, img_cols, nb_channels = 32, 32, 3
nb_classes = 10

# In case depth is 1 (black and white pictures) -> reshape to proper format
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, nb_channels)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, nb_channels)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# convert to one-hot encoding
Y_train = to_categorical(Y_train_scalar)
Y_test = to_categorical(Y_test_scalar)

print('X_train shape:', X_train.shape)
print('Y_train_scalar shape:', Y_train_scalar.shape)
print('Y_train shape (one hot encoding):', Y_train.shape)
print('There is {} train data'.format(X_train.shape[0]))
print('There is {} test data'.format(X_test.shape[0]))
print(img_rows, img_cols, nb_channels)

For computational reasons, we are going to reduce the amount of training data :

In [ ]:
n_train_samples = 10000
X_train = X_train[0:n_train_samples,:,:,:]
Y_train = Y_train[0:n_train_samples,:]
print(X_train.shape)
print(Y_train.shape)



We are now going to create the model described just above, and train it on the previously loaded CIFAR dataset.

## We define the parameters of the model, and of the training

In [ ]:

learning_rate = 0.01
n_epochs = 20
batch_size = 64
# number of convolutional filters to use
nb_filters = 32 

# convolution kernel size
kernel_size = (3, 3)
# size of pooling area for max pooling
pool_size = (2, 2)


# --- Size of the successice layers
n_h_0 = nb_channels
n_h_1 = nb_filters
n_h_2 = nb_filters
n_h_3 = nb_filters

input_shape = (img_rows, img_cols, nb_channels)


Now, define the model :

In [ ]:
# --- START CODE HERE
model = ...

# --- END CODE HERE

Again, compile, display and train the model

In [ ]:
# START CODE HERE
# END CODE HERE

Evaluate the performances of the model on the test data




In [ ]:
score = model.evaluate(X_test, Y_test, verbose=False)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

The train accuracy after 20 epochs should be around 0.4-0.5 (on the test data).

I guess you may be thinking "0.5, that's not very good, what's the deal ? I am pretty disappointed". Some of you may have even found accuracies much lower, depending on your luck with the initialisation.

Well 0.5 is not great, its true, but it's not that bad considering there are 10 categories. A random choice would give an accuracy of 0.1. However, there are several points to consider here :

- The data is more complex than that of mnist for example : there are a lot more variations of "dog"s than "1"s. Therefore, the network has to be larger
- The resolution is  32$\times$32, therefore even for humans it is difficult to recognise some of the images
- Unfortunately, when we start dealing with larger networks, the possibilities for falling into local minima become more and more present. There is no way around this more the moment, due to lack of theoretical understanding. If you do have an idea of how to do this, please email me and we will most likely become very rich.



### Inspecting the network's results

Let's take a look at what the network has learned. What do you think ?

In [ ]:
plt.figure(figsize=(10, 6))
for idx,i in enumerate(range(100,110)):
    plt.subplot(2, 5, idx+1)
    rand_ind = np.random.randint(0,X_test.shape[0])
    predicted_class = ... # FILL IN CODE
    plt.imshow(X_test[rand_ind,:,:,:])
    plt.title(cifar_10_list[int(predicted_class)])

## Exploring the model

You might want to look around inside the network to see what convolutional filters have been learned.

You can explore the network parameters (relatively) easily with Tensorflow. For example ```model.layers``` is a list of the layers of the network. Each element of the network contains the information necessary for this layer. To show the content of a layer i, you can type :

```dir(model.layers[i])```

Let us try to inspect the weights (parameters) of the convolutional network. You can do this in a similar fashion to the way you did this in the RNN lab work (when you displayed the embedding matrix), using :

- layers
- get_weights()

 Note that the get_weights() function returns the following :
- [weights, biases] if the layer is convolutional. weights has size [m,n,filter_depth,n_filters]
- an empty list otherwise

Now, display below all the filter weights of the first layer and for the first channel as images (plt.imshow). Careful, the first layer can be either the ```Input``` or a convolutional layer, depending on which API you used :

In [ ]:

plt.figure(figsize=(10, 6))
for num in range(0,32):
    plt.subplot(8, 4, num+1)
    plt.imshow(...) # FILL IN CODE

What do you think ? Pretty incomprehensible no ? Do not spend too long trying to interpret these filters, that way madness lies. Indeed, they say that you can find some researchers locked in their offices staring at convolutional filters, trying to find some semblance of meaning ...

So, it seems that understanding a CNN by just looking at the filters is an exercise in futility. How can we do better ? Well, take a look at part 2 of the lab !

# Evaluation

To evaluate the work, you should rate the code for 
- 1) MNIST : Defining the first model : 3 points
- 2) MNIST : Compiling and training the first model : 1 point
- 3) CIFAR10 : Defining the second model : 3 points
- 4) CIFAR10 : Compiling and training the first model : 1 point
- 5) Calculating ```predicted_class``` : 1 point
- 6) Correctly visualising the weights : 1 point


Total over 10 points. For the questions with three points :
- 1 point if partially correct
- 2 points if correct code but does not give the expected results or if the code does not compile. Be careful, sometimes the training might get stuck quickly by bad luck : restart the code a few times to make sure it is indeed a bug.
- 3 points for correct code and correct execution